<a href="https://colab.research.google.com/github/41371108H/114-1repo/blob/main/Gradio_%E5%94%AE%E7%A5%A8%E7%B3%BB%E7%B5%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pandas as pd
import gspread
from google.colab import auth
from google.auth import default
import gradio as gr

# --- 認證 Google Colab 環境 ---
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# --- 展覽代號與資訊 ---
exhibitions = {
    'A': {'name': '台北國際電玩展', 'price': 300},
    'B': {'name': '台灣文博會', 'price': 250},
    'C': {'name': '設計師週', 'price': 400},
    'D': {'name': '台北車展', 'price': 350}
}

# --- 連線到 Google 試算表 ---
try:
    gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1YmuPBSwZVq0m0-lwpsjCEcyXik3WVeE7a23sRin5ojw/edit?usp=sharing')
    worksheet = gsheets.worksheet('工作表1')
    sheets_data = worksheet.get_all_values()
    df = pd.DataFrame(sheets_data[1:], columns=sheets_data[0])

    # 確保數字欄位轉換正確
    if not df.empty:
        df['單價'] = df['單價'].astype(int)
        df['購買張數'] = df['購買張數'].astype(int)
        df['購買款項 (總價)'] = df['購買款項 (總價)'].astype(int)

except Exception as e:
    print("⚠️ 載入試算表失敗：", e)
    worksheet = None
    df = pd.DataFrame(columns=['姓名','購買日期','展覽名稱','單價','購買張數','購買款項 (總價)'])

# --- Gradio 互動函式 ---
def purchase_ticket(name, date, exhibition_code, num_tickets):
    global df
    try:
        # 驗證日期
        datetime.datetime.strptime(date, "%Y-%m-%d")
    except:
        return "❌ 日期格式錯誤 (YYYY-MM-DD)", df

    # 驗證代號
    if exhibition_code not in exhibitions:
        return "❌ 無效的展覽代號", df

    # 驗證張數
    try:
        num_tickets = int(num_tickets)
        if num_tickets <= 0:
            return "❌ 張數必須大於 0", df
    except:
        return "❌ 張數必須為數字", df

    info = exhibitions[exhibition_code]
    total_price = info['price'] * num_tickets

    record = {
        '姓名': name,
        '購買日期': date,
        '展覽名稱': info['name'],
        '單價': info['price'],
        '購買張數': num_tickets,
        '購買款項 (總價)': total_price
    }

    # 寫入 Google Sheet
    if worksheet:
        try:
            worksheet.append_row(list(record.values()))
        except Exception as e:
            return f"⚠️ 寫入 Google Sheet 失敗：{e}", df

    # 更新本地 DataFrame
    df = pd.concat([df, pd.DataFrame([record])], ignore_index=True)

    summary = f"✅ 購買成功！\n- 姓名: {name}\n- 展覽: {info['name']}\n- 張數: {num_tickets}\n- 總金額: ${total_price}"
    return summary, df

# --- Gradio 介面 ---
with gr.Blocks() as demo:
    gr.Markdown("## 🎟️ 展覽售票系統 (Gradio 版本)")
    gr.Markdown("請輸入購票資訊：")

    with gr.Row():
        name = gr.Textbox(label="姓名")
        date = gr.Textbox(label="購票日期 (YYYY-MM-DD)")
    with gr.Row():
        exhibition_code = gr.Dropdown(choices=list(exhibitions.keys()), label="展覽代號 (A/B/C/D)")
        num_tickets = gr.Number(label="購買張數", precision=0)

    submit = gr.Button("購買")
    output_msg = gr.Textbox(label="結果")
    output_df = gr.Dataframe(label="目前售票紀錄", interactive=False)

    submit.click(purchase_ticket,
                 inputs=[name, date, exhibition_code, num_tickets],
                 outputs=[output_msg, output_df])

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b26a778091f32f6ef1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
